In [1]:
import os
import numpy as np
from tqdm import tqdm
import csv

In [13]:
filenames_path = '../data/dataset_filenames/'
new_dataset_path_length20 = '../data/new_dataset_length20/'
new_dataset_path = '../data/new_dataset/'

songs_path = '../data/songs_word_level/'

if not os.path.exists(new_dataset_path):
    os.makedirs(new_dataset_path)
if not os.path.exists(new_dataset_path_length20):
    os.makedirs(new_dataset_path_length20)

In [3]:
train_filenames = np.load(filenames_path + 'train_filename_list.pkl', allow_pickle=True)
val_filenames   = np.load(filenames_path + 'valid_filename_list.pkl', allow_pickle=True)
test_filenames  = np.load(filenames_path + 'test_filename_list.pkl',  allow_pickle=True)

In [21]:
def generate_csv(file_names, out_file):
    with open(out_file, 'w') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerow(['filename', 'lyrics', 'syl_lyrics', 'midi_notes'])
        rows = []
        known_songs = []
        for file_name in tqdm(file_names):
            if file_name in known_songs:
                continue
            song_matrix = np.load(songs_path + file_name, allow_pickle=True)[0]
            lyrics = ''
            syl_lyrics = ''
            midi_notes = []
            for i, syllables in enumerate(song_matrix[3]):
                for j, syl in enumerate(syllables):
                    lyrics += syl
                    syl_lyrics += syl + ' '
                    midi_notes.append(song_matrix[1][i][j])
                lyrics += ' '
                
            lyrics = lyrics[:-1]
            syl_lyrics = syl_lyrics[:-1]
            rows.append([file_name, lyrics, syl_lyrics, midi_notes])
            known_songs.append(file_name)
        
        csvwriter.writerows(rows)

def generate_csv_length20(file_names, out_file):
    with open(out_file, 'w') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerow(['filename', 'lyrics', 'syl_lyrics', 'midi_notes'])
        rows = []
        known_songs = []
        for file_name in tqdm(file_names):
            start_idx = 0
            end_idx = 20
            if file_name in known_songs:
                start_idx = 20
                end_idx = 40

            song_matrix = np.load(songs_path + file_name, allow_pickle=True)[0]
            lyrics = ''
            syl_lyrics = ''
            midi_notes = []
            i = start_idx
            count = 0
            for syllables in song_matrix[3][start_idx:end_idx]:
                for j, syl in enumerate(syllables):
                    lyrics += syl
                    syl_lyrics += syl + ' '
                    midi_notes.append(song_matrix[1][i][j])
                    count += 1
                lyrics += ' '
                i += 1
                if count == 20:
                    break

                
            lyrics = lyrics[:-1]
            syl_lyrics = syl_lyrics[:-1]
            rows.append([file_name, lyrics, syl_lyrics, midi_notes])
            known_songs.append(file_name)
        
        csvwriter.writerows(rows)

In [23]:
generate_csv_length20(train_filenames, new_dataset_path_length20 + 'train_length20.csv')
generate_csv_length20(val_filenames,   new_dataset_path_length20 + 'valid_length20.csv')
generate_csv_length20(test_filenames,  new_dataset_path_length20 + 'test_length20.csv')

100%|██████████| 1051/1051 [00:00<00:00, 1472.77it/s]


In [12]:
generate_csv(train_filenames, new_dataset_path + 'train.csv')
generate_csv(val_filenames,   new_dataset_path + 'valid.csv')
generate_csv(test_filenames,  new_dataset_path + 'test.csv')

100%|██████████| 1051/1051 [00:00<00:00, 2700.64it/s]
